In [1]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install -q bitsandbytes
!pip install -q accelerate transformers peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 29.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.3 MB/s eta 0:00:00:00:0100:01


In [3]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
bigframes 2.8.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.8.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible

In [4]:
from collections import Counter
from tqdm.auto import tqdm
import numpy as np
from collections import defaultdict
import re, json, torch, os
import emoji
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import torch
from sklearn.utils.class_weight import compute_class_weight
from datasets import Dataset
import evaluate
from tqdm import tqdm
import random

2025-09-13 13:44:48.642458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757771088.887343      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757771088.973684      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
df_train = pd.read_csv('/content/train.csv')

In [ ]:
df_train.head(5)

,text
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n..."
1,"футболка хорошего качества,но футболка не как ..."
2,Все отлично!!!
3,"Рисунок не очень чёткий, а ткань прозрачная, в..."
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...


# Проведу быстрый обзор, и покажу статистики

In [ ]:
def quick_eda(df, text_col='text', n_samples=5):
  print('Row, cols:', df.shape)
  print('Missing text:', df[text_col].isna().sum())

  df['_len_words'] = df[text_col].fillna('').str.split().str.len()
  df['_len_chars'] = df[text_col].fillna('').str.len()

  print('\nTop 5 shortest examples:')
  display(df.nsmallest(5, '_len_words')[[text_col, '_len_words']])
  print('\nTop 5 longest examples:')
  display(df.nlargest(5, '_len_words')[[text_col, '_len_words']])

  dup = df.duplicated(subset=[text_col]).sum()
  print(f'\nDuplicated text rows: {dup}')

  toks = Counter()
  for t in df[text_col].dropna().astype(str).sample(min(2000, len(df))):
    toks.update(t.lower().split())
  print('\nTop tokens (sampled):', toks.most_common(30)[:30])

In [ ]:
quick_eda(df_train)

Row, cols: (1818, 1)
Missing text: 0

Top 5 shortest examples:


,text,_len_words
102,Отлично!,1
305,Рекомендую.,1
395,Гавно,1
488,огонь!,1
567,Спасибо.,1



Top 5 longest examples:


,text,_len_words
697,"Брюки классные, полоса из кожзама, но мне прис...",41
17,"Товар так и не пришел. Заказала 6 июля, сейчас...",40
740,"ждала 60 с лишним дней, открыла спор чтобы хот...",40
26,"Я думала, что свитер будет более толстым , а н...",39
644,за 300 р. норм халат! на об 109 ог 109 хххл в ...,39



Duplicated text rows: 0

Top tokens (sampled): [('не', 1238), ('на', 795), ('и', 541), ('очень', 452), ('в', 398), ('но', 305), ('размер', 299), ('как', 194), ('товар', 187), ('качество', 187), ('по', 183), ('деньги', 175), ('с', 160), ('так', 155), ('за', 155), ('продавец', 144), ('что', 135), ('а', 133), ('все', 127), ('заказ', 121), ('я', 119), ('ткань', 106), ('доставка', 106), ('к', 101), (',', 99), ('соответствует', 96), ('рост', 87), ('заказала', 85), ('платье', 85), ('пришла', 76)]


*Top-токены показывают «размер», «качество», «доставка», «продавец» → видно, что будет много случаев «нет товара» (например, про доставку/споры).*

Минимальная очистка текста

In [ ]:
URL_RE = re.compile(r'https?://\S+')
HTML_RE = re.compile(r'<[^>]+>')
EMAIL_RE = re.compile(r'\S+@\S+')
PHONE_RE = re.compile(r'\+?\d[\d\-\s]{5,}\d')

def clean_text(t: str) -> str:
  if pd.isna(t):
    return ''
  t = str(t)
  t = t.strip()
  t = URL_RE.sub(' ', t)
  t = HTML_RE.sub(' ', t)
  t = EMAIL_RE.sub(' ', t)
  t = PHONE_RE.sub(' ', t)
  t = emoji.replace_emoji(t, replace=' ')
  t = re.sub(r'\s+', ' ', t)
  return t

In [ ]:
df_train['text_clean'] = df_train['text'].apply(clean_text)

In [ ]:
df_train.head(10)

,text,_len_words,_len_chars,text_clean
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n...",31,205,"Заказали 14.10.2017 , получили 25.10.2017 На м..."
1,"футболка хорошего качества,но футболка не как ...",34,203,"футболка хорошего качества,но футболка не как ..."
2,Все отлично!!!,2,14,Все отлично!!!
3,"Рисунок не очень чёткий, а ткань прозрачная, в...",10,63,"Рисунок не очень чёткий, а ткань прозрачная, в..."
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...,9,75,плохо!!!Низ рваный..деньги не вернули!Открыла ...
5,"обычная майка с ужасным запахом, есть косяки н...",12,72,"обычная майка с ужасным запахом, есть косяки н..."
6,"Все как на фото, и цвет и форма.",8,32,"Все как на фото, и цвет и форма."
7,"Но рост 180 по колено,качество хорошее,доставк...",7,55,"Но рост 180 по колено,качество хорошее,доставк..."
8,Не подошло по размеру. Ткань тоже не понравилась,8,48,Не подошло по размеру. Ткань тоже не понравилась
9,"очень крутой топ,заказала 11.11 пришел 28.11 е...",10,64,"очень крутой топ,заказала 11.11 пришел 28.11 е..."


In [ ]:
kw = {
    "бытовая техника": [
        "пылесос", "утюг", "чайник", "микроволнов", "холодильник", "стирал", "посудомойк",
        "фен", "блендер", "тостер", "соковыжимк", "кондиционер", "обогревател", "пароварк"
    ],
    "обувь": [
        "кроссовк", "кросовк", "кеды", "кед", "ботинк", "ботинок", "туфл", "сапог","сандал",
        "сланц", "сланцы", "тапочк", "тапки", "шлепанц", "шлепанцы", "подошв", "стельк", "шнурк",
        "шнурки", "каблук", "каблуки"
    ],
    "одежда": [
        "футболк", "платье", "брюк", "свитер", "худи", "куртк", "рубашк", "майк", "шорты",
        "толстовк", "топ", "кофт", "джемпер", "шапк", "перчатк", "шарф", "носок", "жилет", "туника"
    ],
    "посуда": [
        "кастрюл", "тарелк", "стакан", "чашк", "ложк", "вилка", "сковород",
        "миск", "кружк", "ковш", "сито", "термос", "чайник"
    ],
    "текстиль": [
        "постель", "полотенц", "штор", "простын", "плед", "покрывало", "подушк", "одеяло",
        "наволочк", "матрас", "одеялц", "пледик", "кухонн"
    ],
    "товары для детей": [
        "памперс", "коляск", "игрушк", "детск", "слинг", "раскраск", "подгузн",
        "кроватк", "стульчик", "горшок", "качел", "машинк", "мобил"
    ],
    "украшения и аксессуары": [
        "кольц", "браслет", "цепочк", "серьг", "сумк", "ремень", "часы", "аксессуар",
        "брошь", "заколк", "подвеск", "ободок", "значок"
    ],
    "электроника": [
        "телефон", "смартфон", "наушник", "зарядк", "планшет", "ноутбук", "камера", "колонк",
        "гарнитур", "монитор", "проектор", "роутер", "мышь", "клавиатур", "тв"
    ],
    "нет товара": [
        "доставка", "упаковк", "продавец", "сервис", "возврат", "обмен", "спор", "курьер",
        "магазин", "оплата", "счёт", "заказ", "доставк", "рекламац"
    ]
}

def keyword_label(text):
  t = text.lower()
  counts = defaultdict(int)
  for cat, keys in kw.items():
    for k in keys:
      if k in t:
        counts[cat] += 1
  if not counts:
    return None, 0.0
  best = max(counts.items(), key=lambda x: x[1])
  values = sorted(counts.values(), reverse=True)
  if len(values) > 1 and values[0] == values[1]:
    return None, 0.0
  return best[0], 0.95

In [ ]:
df_train[['pred_category', 'pred_confidence']] = df_train['text_clean'].apply(lambda x: pd.Series(keyword_label(x)))

In [ ]:
df_train.head(25)

,text,_len_words,_len_chars,text_clean,pred_category,pred_confidence
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n...",31,205,"Заказали 14.10.2017 , получили 25.10.2017 На м...",NaN,0.00
1,"футболка хорошего качества,но футболка не как ...",34,203,"футболка хорошего качества,но футболка не как ...",NaN,0.00
2,Все отлично!!!,2,14,Все отлично!!!,NaN,0.00
3,"Рисунок не очень чёткий, а ткань прозрачная, в...",10,63,"Рисунок не очень чёткий, а ткань прозрачная, в...",NaN,0.00
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...,9,75,плохо!!!Низ рваный..деньги не вернули!Открыла ...,нет товара,0.95
5,"обычная майка с ужасным запахом, есть косяки н...",12,72,"обычная майка с ужасным запахом, есть косяки н...",одежда,0.95
6,"Все как на фото, и цвет и форма.",8,32,"Все как на фото, и цвет и форма.",NaN,0.00
7,"Но рост 180 по колено,качество хорошее,доставк...",7,55,"Но рост 180 по колено,качество хорошее,доставк...",нет товара,0.95
8,Не подошло по размеру. Ткань тоже не понравилась,8,48,Не подошло по размеру. Ткань тоже не понравилась,NaN,0.00
9,"очень крутой топ,заказала 11.11 пришел 28.11 е...",10,64,"очень крутой топ,заказала 11.11 пришел 28.11 е...",нет товара,0.95


In [ ]:
df_train['pred_category'].value_counts()

,count
pred_category,
нет товара,494
электроника,231
одежда,192
украшения и аксессуары,2
обувь,2
посуда,2
текстиль,1
бытовая техника,1


In [ ]:
df_train['pred_category'].isna().sum()

np.int64(893)

In [ ]:
df_train['pred_category'].unique()

array([nan, 'нет товара', 'одежда', 'электроника',
       'украшения и аксессуары', 'текстиль', 'обувь', 'посуда',
       'бытовая техника'], dtype=object)

Как окажется далее, кандидатов на категорию ОБУВЬ и  окажется совсем мало, чтобы это доказать приведу пример здесь, что их по факту оказывается всего один (для ОБУВИ; индекс = 853)

In [ ]:
import re, math
from tqdm import tqdm

# === 1) Найти кандидатов по стемам "обувь"
shoe_stems = ["кроссов", "кросов", "кеды", "кед", "ботин", "ботинок",
              "туф", "сапог", "сандал", "сланц", "тапоч", "тапки",
              "шлеп", "мокас", "подошв", "стельк", "шнурк", "шнурки",
              "босонож", "лофер", "мокасин"]
shoe_pattern = re.compile("|".join([re.escape(s) + r"\w*" for s in shoe_stems]), flags=re.IGNORECASE)

candidates_mask = df_train['text_clean'].fillna("").str.contains(shoe_pattern)
candidates = df_train[candidates_mask].copy().reset_index()
print("Найдено кандидатов (по ключам):", len(candidates))
display(candidates[['index', 'text_clean']].head(30))

Найдено кандидатов (по ключам): 4


,index,text_clean
0,718,"Качество молнии и шнурков плохое, материал не ..."
1,853,Маленькие. Когда одеваешь рисунка почти нет. Б...
2,930,"очень широкие рукава, размер М на мой С велик ..."
3,1291,"качество туфта, размер маленький. Они не стоят..."


# Парсинг, промт - Авторазметка LLM

In [ ]:
categories = [
  {"id": 0, "name": "бытовая техника", "desc": "Пылесосы, чайники, утюги, микроволновки и т.п."},
  {"id": 1, "name": "обувь", "desc": "Кроссовки, ботинки, туфли, сандалии и т.п."},
  {"id": 2, "name": "одежда", "desc": "Футболки, платья, брюки, свитеры и т.п."},
  {"id": 3, "name": "посуда", "desc": "Тарелки, кастрюли, чашки, столовые приборы и т.п."},
  {"id": 4, "name": "текстиль", "desc": "Постельное белье, полотенца, пледы, шторы и т.п."},
  {"id": 5, "name": "товары для детей", "desc": "Игрушки, детская одежда, коляски, памперсы и т.п."},
  {"id": 6, "name": "украшения и аксессуары", "desc": "Кольца, браслеты, сумки, ремни, шарфы и т.п."},
  {"id": 7, "name": "электроника", "desc": "Телефоны, наушники, зарядные, ноутбуки, планшеты и т.п."},
  {"id": 8, "name": "нет товара", "desc": "Отзывы про доставку, упаковку, сервис, возврат или без упоминания конкретного товара."}
]

os.makedirs("configs", exist_ok=True)
with open("configs/categories.json", "w", encoding="utf-8") as f:
    json.dump({"categories": categories}, f, ensure_ascii=False, indent=2)

print("Saved configs/categories.json")

Saved configs/categories.json


In [ ]:
MODEL_NAME = "Qwen/Qwen2-7B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)

id_name_list = "\n".join([f"{c['id']} — {c['name']}: {c['desc']}" for c in categories])

SYSTEM = """Ты ассистент по классификации отзывов.
Твоя задача — отнести отзыв к одной из товарных категорий.
Если отзыв не содержит описания конкретного товара, а говорит только про доставку, оплату, возврат или сервис — тогда выбери категорию "нет товара".
Всегда отвечай строго JSON: {"label": <id категории>, "confidence": <0-100>}"""

USER_TEMPLATE = """
Категории:
{id_name_list}

Отзыв:
"{review}"

Требования:
- Выбери ровно один id категории из списка.
- Если отзыв описывает товар, всегда выбирай соответствующую товарную категорию.
- категорию 8 (нет товара) использовать только если НЕТ упоминания конкретного товара.
- Если отзыв про доставку, упаковку или сервис — выбирай категорию 8 (нет товара).
- Ответ только JSON.
"""

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [ ]:
def aggregate_llm_votes(votes, rare_boost=None, min_conf=50.0):
    from collections import defaultdict
    rare_boost = rare_boost or []

    if not votes:
        return {"label": 8, "confidence": 20.0, "votes": {}, "raw": []}

    label_scores = defaultdict(list)
    for v in votes:
        lab = v.get("label", 8)
        conf = float(v.get("confidence", 50))
        if conf >= min_conf:
            label_scores[lab].append(conf)

    if not label_scores:
        return {"label": 8, "confidence": 25.0, "votes": {}, "raw": votes}

    agg_scores = {}
    for lab, confs in label_scores.items():
        base = sum(confs) / len(confs)
        if lab in rare_boost:
            base *= 1.2
        agg_scores[lab] = base

    best_label = max(agg_scores, key=agg_scores.get)
    best_conf = agg_scores[best_label]

    return {
        "label": best_label,
        "confidence": min(100.0, best_conf),
        "votes": {lab: len(confs) for lab, confs in label_scores.items()},
        "raw": votes
    }

In [ ]:
def extract_json(text):
    try:
        matches = re.findall(r"\{.*?\}", text, re.S)
        if not matches:
            return None
        snippet = matches[-1]  # последний блок в {} обычно правильный
        return json.loads(snippet)
    except Exception:
        return None

bad_cases = []

def run_llm(review, n_votes, temperature, top_p=0.9, max_new_tokens=64):
    prompt = USER_TEMPLATE.format(id_name_list=id_name_list, review=review)
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": prompt}
    ]
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

    results = []
    for _ in range(n_votes):
        out = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )
        decoded = tokenizer.decode(out[0], skip_special_tokens=True)
        parsed = extract_json(decoded)
        if parsed:
            results.append(parsed)
        else:
            bad_cases.append({"review": review, "decoded": decoded})

    if not results:
        return {"label": 8, "confidence": 20.0, "votes": {}, "raw": []}

    # 🔧 новая агрегация
    return aggregate_llm_votes(results, rare_boost=[6, 0, 3, 1])

In [ ]:
"""
def keyword_label(text, kw_dict):
    t = text.lower()
    counts = Counter()
    for cat, keys in kw_dict.items():
        for k in keys:
            if k in t:
                counts[cat] += 1
    if not counts:
        return None, 0.0
    best_cat, best_score = counts.most_common(1)[0]
    return best_cat, 95.0  # keyword считаем "почти железным"

name2id = {c["name"]: c["id"] for c in categories}

def ensemble_label(text, kw_dict, tau_llm=75.0, kw_conf=95.0, n_votes):
    label_kw, conf_kw = keyword_label(text, kw_dict)
    id_kw = name2id.get(label_kw) if label_kw else None

    # Решаем, звать ли LLM
    need_llm = (id_kw is None) or (label_kw == "нет товара")
    llm_res = run_llm(text, n_votes=n_votes) if need_llm else None
    id_llm = llm_res.get("label") if llm_res else None
    conf_llm = llm_res.get("confidence", 0.0) if llm_res else 0.0

    # keyword даёт товарный класс (не "нет товара")
    if id_kw is not None and label_kw != "нет товара":
        if id_llm is not None and id_llm != id_kw:
            return {
                "label": id_llm,
                "confidence": conf_llm,
                "method": "conflict",
                "label_kw": id_kw,
                "label_llm": id_llm,
                "conf_kw": conf_kw,
                "conf_llm": conf_llm
            }
        return {
            "label": id_kw,
            "confidence": kw_conf,
            "method": "keyword",
            "label_kw": id_kw,
            "conf_kw": conf_kw
        }

    # keyword == "нет товара"
    if id_kw is not None and label_kw == "нет товара":
        if id_llm is not None:
            if conf_llm >= tau_llm:
                return {
                    "label": id_llm,
                    "confidence": conf_llm,
                    "method": "llm_confirm_from_kw_noitem",
                    "label_kw": id_kw,
                    "label_llm": id_llm,
                    "conf_llm": conf_llm
                }
            else:
                return {
                    "label": id_kw,
                    "confidence": conf_llm,
                    "method": "kw_noitem_lowconf",
                    "label_kw": id_kw,
                    "label_llm": id_llm,
                    "conf_llm": conf_llm
                }
        else:
            return {
                "label": id_kw,
                "confidence": 50.0,
                "method": "kw_noitem_lowconf",
                "label_kw": id_kw
            }

    # keyword отсутствует
    if id_kw is None:
        if id_llm is not None:
            if conf_llm >= tau_llm:
                return {
                    "label": id_llm,
                    "confidence": conf_llm,
                    "method": "llm",
                    "label_llm": id_llm,
                    "conf_llm": conf_llm
                }
            else:
                return {
                    "label": id_llm,
                    "confidence": conf_llm,
                    "method": "llm_lowconf",
                    "label_llm": id_llm,
                    "conf_llm": conf_llm
                }
        else:
            return {
                "label": None,
                "confidence": 0.0,
                "method": "noinfo"
            }

    return {"label": None, "confidence": 0.0, "method": "fallback"}
"""

Проверка на 10 примерах
Подебажим

In [ ]:
for i in range(150):
    review = df_train.loc[i, "text_clean"]
    res = run_llm(review, temperature=0.9, n_votes=1)  # 1 для быстроты
    print(i, review[:60], "=>", res)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


0 Заказали 14.10.2017 , получили 25.10.2017 На мой размер 42,  => {'label': 3, 'confidence': 100.0, 'votes': {3: 1}, 'raw': [{'label': 3, 'confidence': 85}]}
1 футболка хорошего качества,но футболка не как для девушек и  => {'label': 2, 'confidence': 95.0, 'votes': {2: 1}, 'raw': [{'label': 2, 'confidence': 95}]}
2 Все отлично!!! => {'label': 8, 'confidence': 99.0, 'votes': {8: 1}, 'raw': [{'label': 8, 'confidence': 99}]}
3 Рисунок не очень чёткий, а ткань прозрачная, видно нижнее бе => {'label': 4, 'confidence': 95.0, 'votes': {4: 1}, 'raw': [{'label': 4, 'confidence': 95}]}
4 плохо!!!Низ рваный..деньги не вернули!Открыла спор,но его по => {'label': 0, 'confidence': 100.0, 'votes': {0: 1}, 'raw': [{'label': 0, 'confidence': 85}]}
5 обычная майка с ужасным запахом, есть косяки на белой надпис => {'label': 3, 'confidence': 100.0, 'votes': {3: 1}, 'raw': [{'label': 3, 'confidence': 95}]}
6 Все как на фото, и цвет и форма. => {'label': 3, 'confidence': 100.0, 'votes': {3: 1}, 'raw': [{'la

KeyboardInterrupt: 

Попробуем
Сделать выборку (например, 200–300 отзывов) и прогнать с n_votes=3, temp=0.8 → посмотреть, как сильно увеличится доля классов 1 и 6.

In [ ]:
import random

# 🎯 Выбираем случайные 250 отзывов из train
sample_idx = random.sample(range(len(df_train)), 250)
df_sample = df_train.iloc[sample_idx].reset_index(drop=True)

sample_results = []

for idx, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
    res = run_llm(row["text_clean"], n_votes=1, temperature=0.5)
    res.update({
        "idx": idx,
        "text_clean": row["text_clean"],
    })
    sample_results.append(res)

sample_df = pd.DataFrame(sample_results)

# ---------- фиксим label ----------
def normalize_label(x):
    if isinstance(x, str):
        if x.strip().lower() == "нет товара":
            return 8
        try:
            return int(x)
        except:
            return 8
    return int(x)

sample_df["label"] = sample_df["label"].apply(normalize_label)

print(sample_df["label"].value_counts())

print("\nСредняя уверенность:")
print(sample_df["confidence"].mean())

print("\nПримеры низкоуверенных:")
print(sample_df[sample_df["confidence"] < 75].head(10)[["text_clean", "label", "confidence"]])


  0%|          | 0/250 [00:00<?, ?it/s]

label
2    111
8     68
5     25
7     19
4     13
3      8
0      6
Name: count, dtype: int64

Средняя уверенность:
95.944

Примеры низкоуверенных:
                                            text_clean  label  confidence
88   Пришел цвет не тот который заказывала. На разм...      8        20.0
135  не пришло, долго мучился со спором, продавец н...      8        20.0
144                                Я довольна заказом.      8        20.0
218  Нуууу, заказ пришёл и ушёл! так как не отслежи...      8        20.0
220  Заказала в июне. До сих пор не получила заказ....      8        20.0


Итоговый прогон

In [ ]:
os.makedirs("outputs", exist_ok=True)

results = []
for idx, row in tqdm(df_train.reset_index(drop=True).iterrows(), total=len(df_train)):
    res = run_llm(row["text_clean"], n_votes=1, temperature=0.9)
    res.update({
        "idx": idx,
        "text_clean": row["text_clean"],
        "len_words": row.get("_len_words"),
        "len_chars": row.get("_len_chars"),
    })
    results.append(res)

pseudo_df = pd.DataFrame(results)

# ---------- фиксим label ----------
def normalize_label(x):
    if isinstance(x, str):
        if x.strip().lower() == "нет товара":
            return 8
        try:
            return int(x)
        except:
            return 8  # fallback
    return int(x)

pseudo_df["label"] = pseudo_df["label"].apply(normalize_label)

# 🔧 Сериализуем сложные поля, чтобы parquet не падал
def serialize(x):
    if isinstance(x, (dict, list)):
        return json.dumps(x, ensure_ascii=False)
    return x

for col in ["votes", "raw"]:
    if col in pseudo_df.columns:
        pseudo_df[col] = pseudo_df[col].apply(serialize)

# ✅ Теперь сохраняем
pseudo_df.to_parquet("outputs/pseudo_labels.parquet", index=False)
print("Сохранили pseudo_df:", pseudo_df.shape)

print("Плохих примеров (не JSON):", len(bad_cases))

print("\nРаспределение по классам:")
print(pseudo_df["label"].value_counts())

print("\nСредняя уверенность:")
print(pseudo_df["confidence"].mean())

print("\nПримеры низкоуверенных:")
print(pseudo_df[pseudo_df["confidence"] < 75].head(15)[["text_clean", "label", "confidence"]])

100%|██████████| 1818/1818 [59:35<00:00,  1.97s/it]

Сохранили pseudo_df: (1818, 8)
Плохих примеров (не JSON): 73

Распределение по классам:
label
2    721
8    540
5    173
7    140
4    104
3     92
0     42
6      4
1      2
Name: count, dtype: int64

Средняя уверенность:
93.67216721672168

Примеры низкоуверенных:
                                            text_clean  label  confidence
4    плохо!!!Низ рваный..деньги не вернули!Открыла ...      8        20.0
49       все супер! доставка месяц) спасибо продавцу))      8        20.0
55   не пришли!!! спор ничего не дал! не берите к э...      8        20.0
77   Шло в Москву очень долго, доставка хреновая. О...      8        20.0
92   Очень маленькая. Размерная таблица не соответс...      8        20.0
117                                Спасибо все подошло      8        20.0
175  ни кофты ни денег только просьба не открывать ...      8        20.0
206  все пришло супер,быстро,качество класс,рекомендую      8        20.0
240  товар не пришел 2 месяца ждала, на сообщение н...      8       

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Сохраните датасет в локальную среду Colab
pseudo_df.to_csv('/content/pseudo_df.csv', index=False)

# Скопируйте файл с локальной среды Colab на Google Диск
!cp '/content/pseudo_df.csv' '/content/drive/MyDrive/MyDataset/'

cp: cannot create regular file '/content/drive/MyDrive/MyDataset/': Not a directory


In [ ]:
pseudo_df.head(15)

,label,confidence,votes,raw,idx,text_clean,len_words,len_chars
0,7,100.0,"{""7"": 1}","[{""label"": 7, ""confidence"": 100}]",0,"Заказали 14.10.2017 , получили 25.10.2017 На м...",31,205
1,2,95.0,"{""2"": 1}","[{""label"": 2, ""confidence"": 95}]",1,"футболка хорошего качества,но футболка не как ...",34,203
2,8,100.0,"{""8"": 1}","[{""label"": 8, ""confidence"": 100}]",2,Все отлично!!!,2,14
3,4,100.0,"{""4"": 1}","[{""label"": 4, ""confidence"": 100}]",3,"Рисунок не очень чёткий, а ткань прозрачная, в...",10,63
4,8,20.0,{},[],4,плохо!!!Низ рваный..деньги не вернули!Открыла ...,9,75
5,2,95.0,"{""2"": 1}","[{""label"": 2, ""confidence"": 95}]",5,"обычная майка с ужасным запахом, есть косяки н...",12,72
6,0,100.0,"{""0"": 1}","[{""label"": 0, ""confidence"": 100}]",6,"Все как на фото, и цвет и форма.",8,32
7,5,100.0,"{""5"": 1}","[{""label"": 5, ""confidence"": 100}]",7,"Но рост 180 по колено,качество хорошее,доставк...",7,55
8,5,80.0,"{""5"": 1}","[{""label"": 5, ""confidence"": 80}]",8,Не подошло по размеру. Ткань тоже не понравилась,8,48
9,2,100.0,"{""2"": 1}","[{""label"": 2, ""confidence"": 100}]",9,"очень крутой топ,заказала 11.11 пришел 28.11 е...",10,64


In [ ]:
df = pd.DataFrame(pseudo_df)

# Сохраняем в CSV
df.to_csv('pseudo_df.csv', index=False, encoding='utf-8')

from IPython.display import FileLink
FileLink('pseudo_df.csv')

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [ ]:
pseudo_df = pd.read_csv('/content/pseudo_df_before_filter.csv')

In [ ]:
pseudo_df['label'].value_counts()

,count
label,
2,721
8,540
5,173
7,140
4,104
3,92
0,42
6,4
1,2


In [ ]:
pseudo_df.shape

(1818, 8)

In [ ]:
pseudo_df[pseudo_df['label'] == 1]

,label,confidence,votes,raw,idx,text_clean,len_words,len_chars
983,1,100.0,"{""1"": 1}","[{""label"": 1, ""confidence"": 95}]",983,получила через месяц в Питер. качество хорошее...,21,112
1257,1,100.0,"{""1"": 1}","[{""label"": 1, ""confidence"": 100}]",1257,Маленькая. Заказала л на 46-48. Очень тяжело с...,16,101


In [ ]:
pseudo_df[pseudo_df['label'] == 6]

,label,confidence,votes,raw,idx,text_clean,len_words,len_chars
1078,6,100.0,"{""6"": 1}","[{""label"": 6, ""confidence"": 95}]",1078,Продавец слукавил. Материал шарфв не является ...,8,66
1274,6,100.0,"{""6"": 1}","[{""label"": 6, ""confidence"": 100}]",1274,"Яркие, милые и качество на высоте!",6,34
1302,6,100.0,"{""6"": 1}","[{""label"": 6, ""confidence"": 95}]",1302,"Заказала шарф 13.10, а пришёл ко мне в Крым он...",32,187
1367,6,100.0,"{""6"": 1}","[{""label"": 6, ""confidence"": 95}]",1367,очень красивая..,2,16


# Устранение дисбаланса (Upsampling классов 0, 1, 6; Аугментация текстов этих классов

In [ ]:
# --- keyword baseline ---
def keyword_label(text, kw_dict):
    text_low = text.lower()
    for cat_name, stems in kw_dict.items():
        for stem in stems:
            if re.search(stem, text_low):
                return (cat_name, 1.0)
    return (None, 0.0)

# --- проверка по ключевым словам ---
def passes_keyword_check(text_variant, target_label, kw_dict):
    cat_name, conf = keyword_label(text_variant, kw_dict)
    if cat_name is None:
        return False
    return name2id.get(cat_name) == target_label

# --- проверка через LLM (более мягкая) ---
def passes_llm_check(text_variant, target_label, n_votes=3, threshold=50.0):
    out = run_llm(text_variant, n_votes=n_votes, temperature=0.8)
    lab = out.get("label")
    conf = float(out.get("confidence", 0.0))
    return (lab == target_label) and (conf >= threshold)

In [ ]:
# --- твои маппинги ---
name2id = {c["name"]: c["id"] for c in categories}
id2name = {v: k for k, v in name2id.items()}

# --- генерация синтетики с вариативными промптами и батчами ---
PROMPT_TEMPLATES = [
    "Сгенерируй {n} коротких отзывов о товаре категории «{target_name}». Обязательно используй хотя бы одно слово: {tokens}.",
    "Напиши {n} отзывов покупателей про {target_name}, вставь хотя бы одно из слов: {tokens}.",
    "Придумай {n} разных отзывов про {target_name}. В каждом упоминай хотя бы одно слово из списка: {tokens}.",
]

def generate_and_filter(class_id, n_samples, kw_dict, batch_size=100):
    target_name = id2name[class_id]
    preserve_tokens = kw_dict[target_name]

    accepted, rejected = [], []

    # делим на батчи
    for start in range(0, n_samples, batch_size):
        n_batch = min(batch_size, n_samples - start)
        prompt_template = random.choice(PROMPT_TEMPLATES)
        prompt = prompt_template.format(
            n=n_batch,
            target_name=target_name,
            tokens=", ".join(preserve_tokens)
        )

        messages = [
            {"role": "system", "content": "Ты ассистент по созданию отзывов."},
            {"role": "user", "content": prompt}
        ]

        inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
        out = model.generate(
            inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=1.0,
            top_p=0.98,
            pad_token_id=tokenizer.eos_token_id
        )
        decoded = tokenizer.decode(out[0], skip_special_tokens=True)
        reviews = [ln.strip() for ln in decoded.splitlines() if ln.strip()]

        for r in reviews:
            ok_kw = passes_keyword_check(r, class_id, kw_dict)
            ok_llm = passes_llm_check(r, class_id)
            if ok_kw or ok_llm:
                accepted.append({
                    "text_clean": r,
                    "label": class_id,
                    "confidence": 90.0,
                    "method": "aug_synthetic"
                })
            else:
                rejected.append({
                    "text_clean": r,
                    "label_expected": class_id,
                    "method": "aug_rejected"
                })

    return pd.DataFrame(accepted), pd.DataFrame(rejected)

In [ ]:
# --- запуск для редких классов ---
GEN_SIZES = {0: 1000, 6: 1500, 1: 2000}  # увеличиваем объем
rare_classes = [0, 1, 6]

all_augmented, all_rejected = [], []

for lbl in rare_classes:
    n_to_generate = GEN_SIZES[lbl]
    print(f"\n=== Обработка класса {lbl} === (генерим {n_to_generate})")
    acc_df, rej_df = generate_and_filter(lbl, n_to_generate, kw, batch_size=100)
    print(f"Class {lbl}: accepted {len(acc_df)}, rejected {len(rej_df)}")
    all_augmented.append(acc_df)
    all_rejected.append(rej_df)


=== Обработка класса 0 === (генерим 1000)
Class 0: accepted 125, rejected 52

=== Обработка класса 1 === (генерим 2000)
Class 1: accepted 335, rejected 101

=== Обработка класса 6 === (генерим 1500)
Class 6: accepted 237, rejected 78


In [ ]:
# Объединяем все принятые синтетические примеры
df_augmented = pd.concat(all_augmented, ignore_index=True)

print("Base shape:", pseudo_df.shape)
print("Aug shape:", df_augmented.shape)

# --- опциональное подрезание синтетики ---
MAX_SYNTH = {1: 150, 6: 150, 0: 100}  # лимиты для редких классов
balanced_aug = []

for lbl, group in df_augmented.groupby("label"):
    if lbl in MAX_SYNTH:
        balanced_aug.append(group.sample(n=min(MAX_SYNTH[lbl], len(group)), random_state=42))
    else:
        balanced_aug.append(group)

df_aug_balanced = pd.concat(balanced_aug, ignore_index=True)

Base shape: (1818, 8)
Aug shape: (697, 4)


In [ ]:
# Финальный датасет
df_aug = pd.concat([pseudo_df, df_aug_balanced], ignore_index=True)

print("Final shape:", df_aug.shape)
print("Class distribution:\n", df_aug["label"].value_counts())

Final shape: (2218, 9)
Class distribution:
 label
2    721
8    540
5    173
6    154
1    152
0    142
7    140
4    104
3     92
Name: count, dtype: int64


In [ ]:
df_aug.to_csv('/content/df_with_auc.csv', index=False)

Чистка датасета + continue балансировки

In [ ]:
import pandas as pd
import re

# 1) Убираем все примеры с confidence < 75
df = df_aug[df_aug["confidence"] >= 75].copy()

# 2) Удаляем номера в начале текста вида "14. " или "3. "
df["text_clean"] = df["text_clean"].str.replace(r"^\d+\.\s*", "", regex=True)

# 3) Проверяем и убираем шумные сэмплы с брендами / англ. словами
stopwords = ["panasonic", "review", "microwave", "oven"]
mask_stopwords = df["text_clean"].str.lower().str.contains("|".join(stopwords))
df = df[~mask_stopwords].copy()

# 4) Балансируем классы — обрезаем 2 и 8
MAX_SAMPLES = {2: 400, 8: 400}
balanced_parts = []

for lbl, group in df.groupby("label"):
    if lbl in MAX_SAMPLES:
        balanced_parts.append(group.sample(n=min(MAX_SAMPLES[lbl], len(group)), random_state=42))
    else:
        balanced_parts.append(group)

df_final = pd.concat(balanced_parts, ignore_index=True)

In [15]:
# перешел с колаба на кагл, поэтому еще раз 
df_final = pd.read_csv('/kaggle/input/df-train-final/df_final_clean.csv')

Некоторое дополнение по очитке датасета

In [16]:
# Удаляем номера в начале текста вида "14. " или "3. "
df_final["text_clean"] = df_final["text_clean"].str.replace(r"^\d+\.\s*", "", regex=True)

# Убираем двойные кавычки в начале и в конце строки
df_final["text_clean"] = df_final["text_clean"].str.replace(r'^["«]+|["»]+$', "", regex=True).str.strip()

# Убираем префиксы "Отзев 7:", "Отзыв 12:", "Отзыв №10:" (с кавычкой или без)
df_final["text_clean"] = df_final["text_clean"].str.replace(
    r'^отз[её]в\s*№?\s*\d+:\s*["«»]*', "", regex=True, flags=re.IGNORECASE
)

# Убираем двойные кавычки/ёлочки в начале и в конце строки
df_final["text_clean"] = df_final["text_clean"].str.replace(r'^[\"«]+', "", regex=True)
df_final["text_clean"] = df_final["text_clean"].str.replace(r'[\"»]+$', "", regex=True)

# Убираем префиксы типа "Assistant: 1." или "Assistant:" 
df_final["text_clean"] = df_final["text_clean"].str.replace(r"^assistant:\s*\d*\.?\s*", "", regex=True, flags=re.IGNORECASE)

# Удаляем строки, содержащие непонятные нелатинизированные символы (китайский, японский и т.п.)
df_final = df_final[~df_final["text_clean"].str.contains(r"[\u4e00-\u9fff\u3040-\u30ff]", regex=True)]

# Убираем промпт-утечки:
# "Напиши 100 отзывов..." или "Сгенерируй 100 коротких отзывов..."
df_final = df_final[~df_final["text_clean"].str.lower().str.startswith(("напиши", "сгенерируй", "придумай"))]

# --- Чистим пробелы и кавычки после всех операций
df_final["text_clean"] = df_final["text_clean"].str.strip()
df_final["text_clean"] = df_final["text_clean"].str.replace(r'^[\"«]+|[\"»]+$', "", regex=True).str.strip()

In [17]:
# Смотрим итоговое распределение
print("Итоговое распределение классов:")
print(df_final["label"].value_counts().sort_index())

# Сохраняем финальный датасет
df_final.to_csv("df_final_deb.csv", index=False)

Итоговое распределение классов:
label
0    125
1    143
2    400
3     92
4    104
5    173
6    144
7    140
8    400
Name: count, dtype: int64


In [36]:
# перешел с колаба на кагл, поэтому еще раз 
df_final = pd.read_csv('/kaggle/input/cool-data/df_final_clean_res.csv')

In [29]:
print("Размер датасета после очистки:", df_final.shape)
print(df_final["label"].value_counts())

Размер датасета после очистки: (1720, 9)
label
2    400
8    400
5    173
6    144
1    142
7    140
0    125
4    104
3     92
Name: count, dtype: int64


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from transformers import TrainingArguments, IntervalStrategy

In [37]:
df_final = df_final[["text_clean", "label"]].dropna()

In [38]:
print("Размер датасета после очистки:", df_final.shape)
print(df_final["label"].value_counts())

Размер датасета после очистки: (1719, 2)
label
2    400
8    399
5    173
6    144
1    142
7    140
0    125
4    104
3     92
Name: count, dtype: int64


In [37]:
train_texts, test_texts, train_labels, test_labels = train_test_split(df_final['text_clean'], 
                                                                      df_final['label'],
                                                                     test_size=0.2, random_state=42, 
                                                                      stratify=df_final['label'])

In [42]:
vectorizer = TfidfVectorizer(
    ngram_range=(1,2), 
    max_features=30000, 
    min_df=2
)

X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

Обучение быстрых моделей: LogisticRegression / LinearSVM. Как бейзлайн

In [43]:
logreg = LogisticRegression(max_iter=300, class_weight="balanced", n_jobs=-1)
logreg.fit(X_train, train_labels)
pred_logreg = logreg.predict(X_test)

In [44]:
print("\n=== Logistic Regression ===")
print("Accuracy:", accuracy_score(test_labels, pred_logreg))
print(classification_report(test_labels, pred_logreg, digits=3))


=== Logistic Regression ===
Accuracy: 0.5523255813953488
              precision    recall  f1-score   support

           0      0.810     0.680     0.739        25
           1      0.808     0.750     0.778        28
           2      0.556     0.562     0.559        80
           3      0.238     0.278     0.256        18
           4      0.545     0.571     0.558        21
           5      0.200     0.229     0.213        35
           6      0.903     0.966     0.933        29
           7      0.244     0.393     0.301        28
           8      0.754     0.537     0.628        80

    accuracy                          0.552       344
   macro avg      0.562     0.552     0.552       344
weighted avg      0.591     0.552     0.565       344



In [45]:
svm = LinearSVC(class_weight="balanced")
svm.fit(X_train, train_labels)
pred_svm = svm.predict(X_test)

In [46]:
print("\n=== Linear SVM ===")
print("Accuracy:", accuracy_score(test_labels, pred_svm))
print(classification_report(test_labels, pred_svm, digits=3))


=== Linear SVM ===
Accuracy: 0.5581395348837209
              precision    recall  f1-score   support

           0      0.783     0.720     0.750        25
           1      0.792     0.679     0.731        28
           2      0.504     0.713     0.591        80
           3      0.231     0.167     0.194        18
           4      0.412     0.333     0.368        21
           5      0.192     0.143     0.164        35
           6      1.000     0.931     0.964        29
           7      0.290     0.321     0.305        28
           8      0.671     0.588     0.627        80

    accuracy                          0.558       344
   macro avg      0.542     0.510     0.521       344
weighted avg      0.559     0.558     0.553       344



Переходим к RuBERT fine-tune

In [39]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, IntervalStrategy

In [40]:
train_df, val_df = train_test_split(
    df_final,
    test_size=0.2,
    random_state=42,
    stratify=df_final["label"]
)

print("Train:", train_df.shape, "\nVal:", val_df.shape)

Train: (1375, 2) 
Val: (344, 2)


In [41]:
print(sorted(train_df["label"].unique()))

[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [42]:
print(sorted(val_df["label"].unique()))

[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [43]:
MODEL_NAME = "cointegrated/rubert-tiny2"
NUM_LABELS = train_df["label"].nunique()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [44]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['label'])
val_df['label'] = le.transform(val_df['label'])

NUM_LABELS = len(le.classes_)
print("NUM_LABELS:", NUM_LABELS)
print("train labels min/max:", train_df['label'].min(), train_df['label'].max())

NUM_LABELS: 9
train labels min/max: 0 8


In [45]:
def preprocess(batch):
  return tokenizer(
      batch["text_clean"],
      truncation=True,
      padding="max_length",
      max_length=128
  )

train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

train_ds = train_ds.map(preprocess, batched=True)
val_ds = val_ds.map(preprocess, batched=True)

train_ds = train_ds.rename_column("label", "labels").with_format("torch")
val_ds = val_ds.rename_column("label", "labels").with_format("torch")

Map:   0%|          | 0/1375 [00:00<?, ? examples/s]

Map:   0%|          | 0/344 [00:00<?, ? examples/s]

In [46]:
classes = np.arange(NUM_LABELS)
weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=train_df["label"].values
)

# Приведём к float32 и проверим размер
weights = np.asarray(weights, dtype=np.float32)
assert len(weights) == NUM_LABELS, f"weights mismatch: {len(weights)} vs {NUM_LABELS}"

class_weights = torch.tensor(weights, dtype=torch.float32, device="cuda")
print("class_weights:", class_weights)

class_weights: tensor([1.5278, 1.3402, 0.4774, 2.0646, 1.8407, 1.1071, 1.3285, 1.3641, 0.4789],
       device='cuda:0')


In [49]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS
)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # Ваши обязательные метрики (добавьте нужные)
    accuracy = accuracy_score(labels, predictions)
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_macro = f1_score(labels, predictions, average='macro')
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'precision': precision,
        'recall': recall
    }

In [55]:
training_args = TrainingArguments(
    output_dir="./rubert_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=1.5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    greater_is_better=True,
    push_to_hub=False,
    report_to="none",
    logging_dir='./logs',
    logging_steps=100
)

In [56]:
from transformers import Trainer
from torch.nn import CrossEntropyLoss

In [59]:
def custom_loss(model, inputs, return_outputs=False, **kwargs):
    """
    Кастомный loss для Trainer с учетом class_weights.
    **kwargs ловит любые дополнительные аргументы Trainer.
    """
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fct = CrossEntropyLoss(weight=class_weights.to(logits.device))
    loss = loss_fct(logits.view(-1, NUM_LABELS), labels.view(-1))
    return (loss, outputs) if return_outputs else loss

In [60]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_36/3457493100.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [61]:
trainer.compute_loss = custom_loss

In [62]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Macro,Precision,Recall
1,1.406800,1.466503,0.627907,0.559856,0.502803,0.563308,0.627907
2,1.316000,1.405731,0.645349,0.595615,0.554395,0.582528,0.645349
3,1.246100,1.359063,0.648256,0.609470,0.574411,0.602903,0.648256
4,1.191100,1.322056,0.645349,0.607879,0.567992,0.597492,0.645349
5,1.141700,1.289110,0.656977,0.627288,0.591523,0.614851,0.656977
6,1.095500,1.263715,0.654070,0.626354,0.592096,0.613606,0.654070
7,1.052200,1.240837,0.654070,0.628521,0.589888,0.623311,0.654070
8,1.025900,1.224106,0.656977,0.631373,0.598039,0.621865,0.656977
9,0.991300,1.210449,0.659884,0.634793,0.600747,0.627586,0.659884
10,0.971300,1.199815,0.662791,0.638011,0.605987,0.629470,0.662791


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=645, training_loss=1.0734370505162911, metrics={'train_runtime': 57.4661, 'train_samples_per_second': 358.907, 'train_steps_per_second': 11.224, 'total_flos': 38057960160000.0, 'train_loss': 1.0734370505162911, 'epoch': 15.0})